## 1. 패키지 임포트 및 데이터 읽어오기

In [4]:
import glob
import tensorflow as tf
import os
import re

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path) # txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list에 할당

raw_corpus = [] 


for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #  문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too ", "But it's just like my girl ", "When she's around me ", 'I just feel so good, so good ', 'But right now I just feel cold, so cold ', 'Right down to my bones ', "'Cause ooh... ", "Ain't no sunshine when she's gone ", "It's not warm when she's away ", "Ain't no sunshine when she's gone ", "And she's always gone too long ", 'Anytime she goes away ']


## 2. 데이터 전처리하기

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    
    return sentence

In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 공백인 문장은 건너뜀.
    if sentence[-1] == ":": continue # 문장의 끝이 : 인 문장은 건너뜁니다.
    if sentence[-1] == "]": continue # 문장의 끝이 ] 인 문장은 건너뜁니다.                               
        
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence( sentence )
    corpus.append( preprocessed_sentence )
    
        
# 정제된 결과를 20개만 확인해보죠
corpus[:20]

['<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>',
 '<start> but right now i just feel cold , so cold <end>',
 '<start> right down to my bones <end>',
 '<start> cause ooh . . . <end>',
 '<start> ain t no sunshine when she s gone <end>',
 '<start> it s not warm when she s away <end>',
 '<start> ain t no sunshine when she s gone <end>',
 '<start> and she s always gone too long <end>',
 '<start> anytime she goes away <end>',
 '<start> wonder this time where she s gone <end>',
 '<start> wonder if she s gone to stay <end>',
 '<start> ain t no sunshine when she s gone <end>']

## 3. 데이터를 토큰화하고, 텐서 만들기

In [7]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 12000단어를 기억할 수 있는 tokenizer
        filters=' ',
        oov_token="<unk>" # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿈.
    )
    tokenizer.fit_on_texts(corpus) # 문자 데이터를 입력받아 리스트의 형태로 변환

    tensor = tokenizer.texts_to_sequences(corpus) # 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환
       
    # 문장 뒤에 패딩을 붙여 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') 

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor, tokenizer)

[[  2   7 156 ...   0   0   0]
 [  2  17   7 ...   0   0   0]
 [  2   6  98 ...   0   0   0]
 ...
 [  2 310   1 ...   0   0   0]
 [  2 729   5 ...   0   0   0]
 [  2 729   5 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f55906c8880>


In [8]:
for idx in tokenizer.index_word: # 단어의 인덱스와 인덱스에 해당하는 단어를 딕셔너리 형으로 반환함. (단어사전)
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : your


## 4. 소스문장과 타겟문장을 생성하고, 데이터셋 만들기

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성함.
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성함.
tgt_input = tensor[:, 1:]

# 훈련 데이터와 평가 데이터 분리
from  sklearn.model_selection  import  train_test_split
enc_train, enc_val, dec_train, dec_val =  train_test_split( src_input, 
                                                            tgt_input,  
                                                            test_size=0.2, 
                                                            random_state=10 )

## 5. 언어 모델의 레이어 구성하기

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding( vocab_size, embedding_size ) # 단어 사전의 인덱스값을 해당 인덱스번째의 워드벡터로 바꿔줌.
        self.rnn_1 = tf.keras.layers.LSTM( hidden_size, return_sequences=True )  
        self.rnn_2 = tf.keras.layers.LSTM( hidden_size, return_sequences=True )
        self.linear = tf.keras.layers.Dense( vocab_size )
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
vocab_size = tokenizer.num_words + 1 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개     
embedding_size = 256 # 워드 벡터의 차원수를 말하며, 단어가 추상적으로 표현될 수 있는 크기임.
hidden_size = 1024 
model = TextGenerator(vocab_size, embedding_size , hidden_size) 

## 6. 언어 모델 훈련하기

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수
    from_logits=True, # 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려줌.
    reduction='none')  # 각자 나오는 값의 반환을 원할 때 None을 사용함.

model.compile(loss=loss, optimizer=optimizer) 
model.fit(enc_train, dec_train, epochs=10)

## 7. 작문을 수행하는 함수 만들기

In [ ]:
def generate_text( model, tokenizer, init_sentence="<start>", max_len=20 ): 
    test_input = tokenizer.texts_to_sequences([init_sentence]) 
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax( tf.nn.softmax( predict, axis=-1 ), axis=-1 )[:, -1]  
          # 예측된 값 중 가장 높은 확률의 word index를 뽑아냄.

        test_tensor = tf.concat([ test_tensor, tf.expand_dims( predict_word, axis=0 )], axis=-1 )
          # 예측된 word index를 문장 뒤에 붙임.

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
         # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침.

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "    # word index를 단어로 하나씩 변환함.
        return generated   # 최종적으로 모델이 생성한 문장을 반환함.

In [ ]:
# generate_text 함수에 모델을 이용해서 ilove 로 시작되는 문장을 생성함.
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

## 8. 언어 모델 평가하기 

In [ ]:
dec_pred = model.predict(enc_val)

from  sklearn.metrics  import  classification_report
print(classification_report( dec_val, dec_pred ))